In [1]:
import torch

In [5]:
m = torch.rand(2, 3); m

tensor([[0.8823, 0.9150, 0.3829],
        [0.9593, 0.3904, 0.6009]])

In [6]:
type(m)

torch.Tensor

In [10]:
m.reshape(1,6)

tensor([[0.2566, 0.7936, 0.9408, 0.1332, 0.9346, 0.5936]])

In [11]:
torch.ones(2, 3)

tensor([[1., 1., 1.],
        [1., 1., 1.]])

In [12]:
torch.ones((2, 3))

tensor([[1., 1., 1.],
        [1., 1., 1.]])

In [13]:
torch.equal(torch.ones(2, 3), torch.ones((2, 3)))

True

In [14]:
torch.ones(2, 3, dtype=torch.int)

tensor([[1, 1, 1],
        [1, 1, 1]], dtype=torch.int32)

In [15]:
torch.Tensor([1, 2, 3])

tensor([1., 2., 3.])

In [16]:
torch.tensor([1, 2, 3])

tensor([1, 2, 3])

In [17]:
torch.Tensor([1, 2, 3]).dtype

torch.float32

In [18]:
torch.tensor([1, 2, 3]).dtype

torch.int64

In [19]:
torch.Tensor([1, 2, 3], dtype=torch.float16)

TypeError: new() received an invalid combination of arguments - got (list, dtype=torch.dtype), but expected one of:
 * (*, torch.device device)
      didn't match because some of the keywords were incorrect: dtype
 * (torch.Storage storage)
 * (Tensor other)
 * (tuple of ints size, *, torch.device device)
 * (object data, *, torch.device device)


In [32]:
torch.tensor([1, 2, 3], dtype=torch.float16)

tensor([1., 2., 3.], dtype=torch.float16)

In [58]:
m = torch.ones((2, 3)); m

tensor([[1., 1., 1.],
        [1., 1., 1.]])

In [59]:
m.ndim

2

In [60]:
m[1, 1]

tensor(1.)

In [61]:
m[1, 1].item()

1.0

In [62]:
m.shape

torch.Size([2, 3])

In [63]:
m = m.reshape(-1, 6); m

tensor([[1., 1., 1., 1., 1., 1.]])

In [64]:
m.squeeze(0)

tensor([1., 1., 1., 1., 1., 1.])

In [65]:
m

tensor([[1., 1., 1., 1., 1., 1.]])

In [66]:
m.squeeze_(0)

tensor([1., 1., 1., 1., 1., 1.])

In [67]:
m

tensor([1., 1., 1., 1., 1., 1.])

In [68]:
m.unsqueeze_(0)

tensor([[1., 1., 1., 1., 1., 1.]])

In [70]:
m

tensor([[1., 1., 1., 1., 1., 1.]])

In [71]:
m + 1

tensor([[2., 2., 2., 2., 2., 2.]])

In [72]:
m.add_(1)

tensor([[2., 2., 2., 2., 2., 2.]])

In [76]:
m.resize_(2, -1)

RuntimeError: Trying to create tensor with negative dimension -1: [2, -1]